In [1]:
import sys, os
from pathlib import Path

sys.path.append(os.path.abspath(Path().resolve().parent/'src'))

In [2]:
from kinematicsrobotics.datahandler import Extract
from kinematicsrobotics.dataprocessing import Preprocessing
from kinematicsrobotics.model import Model
from kinematicsrobotics.metrics import Metrics
from kinematicsrobotics.kinematics import Robo,Spacemapping

## Dataset

In [3]:
ext = Extract()
ext._path_project = os.path.abspath(Path().resolve().parent)

dataset = ext.dataframe( r'src\data\ready\dataset-radius-1cm.csv')
dataset

,theta_1,theta_2,theta_3,theta_4,theta_5,p_x,p_y,p_z,roll,pich,yaw
0,0,0,12,0,0,53.213314,-2.477121e-15,2.515179,3.141593e+00,-1.361357,2.752040e-16
1,0,12,12,0,0,53.606657,-1.789652e-15,13.742410,2.378531e+00,-1.570796,7.630613e-01
2,0,12,24,0,0,52.819970,-3.124530e-15,6.257590,3.141593e+00,-1.361357,2.371518e-16
3,0,12,24,12,0,53.213314,-2.915404e-15,10.000000,2.356194e+00,-1.570796,7.853982e-01
4,0,12,36,12,0,51.657132,-4.201082e-15,2.678740,3.141593e+00,-1.361357,2.500234e-16
...,...,...,...,...,...,...,...,...,...,...,...
6884,120,120,120,60,0,-9.000000,1.558846e+01,41.176915,-2.026424e-16,-0.523599,-1.047198e+00
6885,120,120,120,72,0,-7.281153,1.261133e+01,42.707475,-1.899993e-16,-0.314159,-1.047198e+00
6886,120,120,120,84,0,-5.440756,9.423666e+00,43.489851,-1.843716e-16,-0.104720,-1.047198e+00
6887,120,120,120,96,0,-3.559244,6.164791e+00,43.489851,-1.843716e-16,0.104720,-1.047198e+00


In [4]:
data  = Preprocessing(dataset = dataset, 
                           x_labels=['p_x', 'p_y','p_z', 'roll', 'pich', 'yaw'],
                           y_labels=['theta_1', 'theta_2', 'theta_3', 'theta_4']
                        )

In [5]:
x_train, x_test, y_train, y_test = data.data_train_test

## Modelo

In [24]:
mlp = Model.mlp_regressor(hidden_layer_sizes=(300,300), EPOCHS_NOCHANGE = 20, activation = 'tanh')

## Estimação de Parâmatros

In [19]:
# Estimação de hiperparâmetros
history = ext.dataframe(r'src\data\ready\history.csv')

history_best = history[history['rank_test_score'] == 1]

params = eval(history_best.iloc[0]['params'])

In [8]:
mlp.set_model(**params)

## Treino

In [25]:
mlp.fit(x=x_train,y=y_train)

## Metricas

In [26]:
metrica = Metrics(model = mlp, 
                  preprocessing = data)

In [27]:
metrica.mse(mlp.predict(x =x_train), y_train)

array([0.00076536, 0.00338363, 0.00833304, 0.00350107])

### Erro juntas

In [10]:
metrica.predict_joint(x = x_train[:1])

array([[73.78544208, 71.85633611, 95.78364584, 36.20498405]])

In [11]:
metrica.mse_joint(x = x_train[:1], y = y_train[:1])

array([1.47515095, 0.02063931, 0.04680912, 0.04201846])

### Erro cartesiano

In [3]:
Elos = [['theta_1',10,0,90,0],
        ['theta_2',0,18,180,0],
        ['theta_3',0,18,-180,0],
        ['theta_4',0,0,90,90],
        ['theta_5',18,0,0,0]
]

robo = Robo("Robo", Elos)



In [28]:
metrica.robotic(robo = robo)

In [8]:
robo.matrixForwardKinematics

Matrix([
[ sin(theta_1)*sin(theta_5) - sin(theta_2 - theta_3 + theta_4)*cos(theta_1)*cos(theta_5), sin(theta_1)*cos(theta_5) + sin(theta_5)*sin(theta_2 - theta_3 + theta_4)*cos(theta_1), cos(theta_1)*cos(theta_2 - theta_3 + theta_4), 18*(cos(theta_2) + cos(theta_2 - theta_3) + cos(theta_2 - theta_3 + theta_4))*cos(theta_1)],
[-sin(theta_1)*sin(theta_2 - theta_3 + theta_4)*cos(theta_5) - sin(theta_5)*cos(theta_1), sin(theta_1)*sin(theta_5)*sin(theta_2 - theta_3 + theta_4) - cos(theta_1)*cos(theta_5), sin(theta_1)*cos(theta_2 - theta_3 + theta_4), 18*(cos(theta_2) + cos(theta_2 - theta_3) + cos(theta_2 - theta_3 + theta_4))*sin(theta_1)],
[                                          cos(theta_5)*cos(theta_2 - theta_3 + theta_4),                                         -sin(theta_5)*cos(theta_2 - theta_3 + theta_4),              sin(theta_2 - theta_3 + theta_4),     18*sin(theta_2) + 18*sin(theta_2 - theta_3) + 18*sin(theta_2 - theta_3 + theta_4) + 10],
[                                    

In [11]:
_mapping = Spacemapping(robo)

In [14]:
# Entrada normalizada
x_train[:1]

array([[-1.38744751,  0.95597318, -0.25828336, -0.33585617, -0.67744555,
        -0.0453049 ]])

In [15]:
# Saída normalizada
y_train[:1]

array([[ 1.63280665, -0.39449589,  0.68512865, -0.05322614]])

In [16]:
# Entrada real
data._scaler_x.inverse_transform(x_train[:1])

array([[-1.98063905e+01,  3.43056746e+01,  3.06978664e+01,
        -1.11022302e-16, -1.15191731e+00, -1.04719755e+00]])

In [17]:
# Saída real (Theta)
data._scaler_y.inverse_transform(y_train[:1])

array([[120.,  72.,  84.,  36.]])

In [18]:
_mapping.operational_space([[120.,  72.,  84.,  36.,0]], output_format='DataFrame')

,R_11,R_12,R_13,p_x,R_21,R_22,R_23,p_y,R_31,R_32,R_33,p_z
0,0.203368,0.866025,-0.456773,-19.80639,-0.352244,0.5,0.791154,34.305675,0.913545,0.0,0.406737,30.697866


In [19]:
# Theta predito
a = metrica.predict_joint(x = x_train[:1])
a

array([[119.34226985,  73.21715963,  85.90174503,  36.08519321]])

In [20]:
from numpy import append

def transfor(array):
            novo_array = []
            for sub_array in array:
                    novo_sub_array = append(sub_array, 0)
                    novo_array.append(novo_sub_array)
            return novo_array

b = transfor(a)[0]
x_predito = [b.tolist()]
x_predito

[[119.34226984559332,
  73.21715962924691,
  85.90174502516061,
  36.08519321273819,
  0.0]]

In [23]:
_mapping.operational_space(x_predito, output_format='DataFrame')

,R_11,R_12,R_13,p_x,R_21,R_22,R_23,p_y,R_31,R_32,R_33,p_z
0,0.204191,0.279235,-0.938261,-27.305081,-0.059379,0.960223,0.272849,7.940383,0.977129,0.0,0.21265,-10.577714


In [62]:
data._scaler_x.inverse_transform(x_train[:1])

array([[-19.22497119,  33.29862687,  12.47649814,   3.14159265,
         -1.15191731,   2.0943951 ]])

#### t

In [24]:
metrica.predict_operacional(x = x_train[:1])

,p_x,p_y,p_z
0,-27.305081,7.940383,-10.577714


In [29]:
metrica.mse_operacional(x = x_test)

(array([0.36797647, 0.25883504, 0.36246946]),
 array([[-10.84131807,  40.46034987,  10.94593812],
        [-16.34014254, -23.336142  ,  47.23226182],
        [-13.68409139,  -4.98060195,  54.81816913],
        ...,
        [ -9.23454387, -25.37170076,  56.7653718 ],
        [-17.34345885,  30.0397519 ,  48.27928638],
        [-10.75711632,  40.14610467,   4.99167727]]),
            p_x        p_y        p_z
 0    -9.948729  40.279144  11.632190
 1   -16.495747 -23.279149  47.468411
 2   -13.553524  -4.636652  56.067567
 3    16.915587   4.300232  47.981086
 4    17.142071  20.510338  49.677894
 ..         ...        ...        ...
 684  -4.927778  29.394514  24.011418
 685  10.793524  42.107433  40.603587
 686  -9.178889 -25.543034  56.679121
 687 -16.588240  30.586090  48.487517
 688 -10.690749  39.340309   6.079594
 
 [689 rows x 3 columns])